In [8]:
def crear_ansantz(n, parametros):
    
    ansantz = QuantumCircuit(n)
        
    for i in range(n):
        
        ansantz.ry(parametros[i], i)
        
    for i in range(n-1, 0, -1):
        
        ansantz.cx(i, i-1)
        
    for i in range(n):
        
        ansantz.ry(parametros[i + n], i)
        
    return ansantz
    
    

In [2]:
def resolver(H, crear_ansantz):
    
    params = np.random.rand(H.n_qubits * 2) * np.pi
    
    def func(params):
        return H.calcular_valor_esperado(crear_ansantz(H.n_qubits, params))
    
    solucion = scipy.optimize.minimize(fun = func, method = "COBYLA", x0 = params)
    
    circ = QuantumCircuit(H.n_qubits, H.n_qubits)
    circ += crear_ansantz(H.n_qubits, solucion['x'])
    circ.measure(range(H.n_qubits), range(H.n_qubits))
    
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circ, backend, shots = 1)
    result = job.result().get_counts()
    return list(result.keys())[0]
    
    

In [3]:
def aclarar_solucion(solucion):
    
    for i, color in enumerate(solucion):
        
        if color == '0':
            print(nodos[i], "es de color rojo")
        else:
            print(nodos[i], "es de color azul")

In [16]:
import hamiltoniano as hmt
from qiskit import *
import scipy
import matplotlib.pyplot as plt
import numpy as np

nodos = ['A', 'B', 'C', 'D', 'E']


pesos = {
    'AB': 1,
    'BA': 1,
    'AC': 4,
    'CA': 4,
    'AD': 2,
    'DA': 2,
    'BE': 3,
    'EB': 3,
    'CD': 2,
    'DC': 2,
    'DE': 5,
    'ED': 5
}

n_qubits = len(nodos)

H = hmt.Hamiltoniano(n_qubits)
for peso in pesos:
    
    termino1 = hmt.Termino_hamiltoniano([['Z', nodos.index(peso[0])],['Z', nodos.index(peso[1])]], 
                                   n_qubits, pesos[peso]/2)
    
    H.incluir_termino(termino1)

    
solucion = resolver(H, crear_ansantz)

aclarar_solucion(solucion)

circ_valor = QuantumCircuit(n_qubits)

for i, color in enumerate(solucion):
    
    if color == '0':
        
        circ_valor.x(i)
        
print("con un coste de", -H.calcular_valor_esperado(circ_valor))

A es de color rojo
B es de color azul
C es de color azul
D es de color azul
E es de color rojo
con un coste de 13.0
